# Load table to a pandas DataFrame

First, we load the table using pyspark into a pandas DataFrame. This makes feature engineering convenient by using the pandas and scikit-learn packages.

In [0]:
# Load data table
df = spark.read.table("energy_clean")
df = df.toPandas()
display(df.head(10))

country,year,MTOE,TOE_HAB
SK,2019,11.2,2.05
BA,2018,4.4,1.26
PL,2018,74.9,1.97
CZ,2011,24.5,2.33
LV,2007,4.4,1.98
DE,2021,208.1,2.5
IT,2015,116.2,1.91
LV,2019,4.1,2.13
TR,2001,50.7,0.78
FI,2022,23.3,4.2


# Feature engineering and training data preparation

The data in the table has already been cleaned and preprocessed in the previous ``energy_preprocessing`` notebook. However, depending on the ML algorithm, some additional feature engineering may be needed. For example, most ML algorithms do not handle strings or categorical columns well, for example the country one. 

Instead of using strings, we can encode the column to integer values. Even if the values are now integers, the values are still categorical, which can be handled by neural networks but may not be optimal for tree based methods. This is because the trees work assuming a logical order between mumerical values, which does not make sense with categorical data (21 (Spain) > 20 (Germany)). For this, we can use one-hot encoding, which creates a column for each country with 0 (not this country) or 1 (yes). The drawback is that this can explode the size of the dataset, depending on the number of categories.

In [0]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder

# One hot encoding, should work best for Random Forests, but increases training data dimensionality
df["country"] = df["country"].astype("category")
df["country_copy"] = df["country"]
df = pd.get_dummies(df, 
                    columns=["country_copy"], 
                    prefix="country",
                    drop_first=True, 
                    dtype=np.int8) # np.uint8 is not supported by pyspark

# Simple encoding which can be used as a categorical feature for XGBoost, more memory efficient
# Will also be used for embedding in the neural network
country_encoder = LabelEncoder()
df.insert(1, "country_encoded", country_encoder.fit_transform(df["country"]), allow_duplicates=True)
display(df.head(10))

country,country_encoded,year,MTOE,TOE_HAB,country_AT,country_BA,country_BE,country_BG,country_CY,country_CZ,country_DE,country_DK,country_EE,country_EL,country_ES,country_FI,country_FR,country_HR,country_HU,country_IE,country_IS,country_IT,country_LT,country_LU,country_LV,country_ME,country_MK,country_MT,country_NL,country_NO,country_PL,country_PT,country_RO,country_RS,country_SE,country_SI,country_SK,country_TR,country_UK,country_XK
SK,33,2019,11.2,2.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
BA,2,2018,4.4,1.26,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PL,27,2018,74.9,1.97,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
CZ,6,2011,24.5,2.33,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
LV,21,2007,4.4,1.98,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DE,7,2021,208.1,2.5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
IT,18,2015,116.2,1.91,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
LV,21,2019,4.1,2.13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TR,34,2001,50.7,0.78,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
FI,12,2022,23.3,4.2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


To demonstrate how well the ML algorithms work, we split the data into the train data used for tuning and training the ML models, and the test data which will only be used to compare which ML model works best. To showcase the model's capabilities of predicting future values, the test data is the last 4 years of data, and will not be used for training.

In [0]:
# For demonstration purposes only, we will take the last 4 years of data for testing purposes
# This allows us to show how well the models generalize temporally
# The final model will be trained on a random 80/20 split as commonly done
df = df.sort_values(by="year", ascending=True)
df_train = df[df["year"] <= 2018].copy()
df_test = df[df["year"] > 2018].copy()


# Machine Learning training

## Random Forests

The first ML algorithm we are going to try is the Random Forest regressor. This works by creating an ensemble of decision trees. Each decision tree is trained on a subset of the data. The trees fit the data by learning simple decision rules. The first node of the tree could be ``x > 2015``, which splits the tree in two nodes based on the answer. The leaves contain the average value for the data that lead to that node. 

``n_estimators`` controls the number of trees and ``max_depth`` the maximum depth of the trees. To tune these parameters, as will be done with the other ML algorithms, we use Cross-validation. This method splits the training data into k splits (k=5 in this case). For each of the chosen parameter combinations, it will train on k-1 sets and test on the remaining set. The parameters that lead to the best overal root mean squared error (RMSE) will be selected. Given that the data is temporally correlated, we use ``TimeSeriesSplit`` which splits the training data based on time instead of randomly.

To see the difference of using one-hot encoding for the countries compared to simple numerical encoding, we tune and train to separate models.

In [0]:
import mlflow
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor

# Test with simple country encoding
target_col = "TOE_HAB"
feature_cols = ["country_encoded", "year"]

X, y = df_train[feature_cols], df_train[target_col]

# Do a simple grid seach to find the best parameters
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20, 30],
}

# Use time series splits for temporally correlated data
tscv = TimeSeriesSplit(n_splits=5)

print()
grid_search = GridSearchCV(RandomForestRegressor(), 
                           param_grid=param_grid, 
                           cv=tscv, 
                           scoring="neg_root_mean_squared_error",
                           verbose=True)
grid_search.fit(X, y)

print("Best Parameters:", grid_search.best_params_)
max_depth = grid_search.best_params_["max_depth"]
n_estimators = grid_search.best_params_["n_estimators"]

# Prepare train and test data
X_train, y_train = np.array(df_train[feature_cols]), np.array(df_train[target_col])
X_test, y_test = np.array(df_test[feature_cols]), np.array(df_test[target_col])
print(f"\nTraining data shapes -> X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")

# Train RandomForestRegressor
print(f"\nTraining random forest regressor")
with mlflow.start_run(run_name="random_forest_regressor_simple"):
    
    # Initialize model
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    
    # Train model
    rf.fit(X_train, y_train)

    # Predict and measure performance
    y_pred = rf.predict(X_test)
    rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Log with MLFlow
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("country_encoding", "simple")
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)

    mlflow.sklearn.log_model(rf, "random_forest_regressor_simple", input_example=X_train[0:1])

print(f"RMSE: {rmse:.3f}, R2: {r2:.3f}")


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters: {'max_depth': 30, 'n_estimators': 200}

Training data shapes -> X_train: (681, 2), y_train: (681,), X_test: (132, 2), y_test: (132,)

Training random forest regressor
RMSE: 0.294, R2: 0.956


In [0]:
# Test with one-hot country encoding, should work betteree
target_col = "TOE_HAB"
feature_cols = ["year"] + [col for col in df.columns if (col.startswith("country_") and not col.endswith("encoded"))]

X, y = df_train[feature_cols], df_train[target_col]

# Do a simple grid seach to find the best parameters
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20, 30],
}

# Use time series splits for temporally correlated data
tscv = TimeSeriesSplit(n_splits=5)

print()
grid_search = GridSearchCV(RandomForestRegressor(), 
                           param_grid=param_grid, 
                           cv=tscv, 
                           scoring="neg_root_mean_squared_error",
                           verbose=True)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
max_depth = grid_search.best_params_["max_depth"]
n_estimators = grid_search.best_params_["n_estimators"]

# Prepare test and train data
X_train, y_train = np.array(df_train[feature_cols]), np.array(df_train[target_col])
X_test, y_test = np.array(df_test[feature_cols]), np.array(df_test[target_col])
print(f"\nTraining data shapes -> X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")

# Train RandomForestRegressor
print(f"\nTraining random forest regressor")
with mlflow.start_run(run_name="random_forest_regressor_one_hot"):

    # Initialize model
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    
    # Train model
    rf.fit(X_train, y_train)

    # Predict and measure performance
    y_pred = rf.predict(X_test)
    rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Log with MLFlow
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("country_encoding", "one_hot")
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)

    mlflow.sklearn.log_model(rf, "random_forest_regressor_one_hot", input_example=X_train[0:1])

print(f"RMSE: {rmse:.3f}, R2: {r2:.3f}")



Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters: {'max_depth': 30, 'n_estimators': 100}

Training data shapes -> X_train: (681, 37), y_train: (681,), X_test: (132, 37), y_test: (132,)

Training random forest regressor
RMSE: 0.317, R2: 0.949


Even though one-hot encoding for the countries should represent better the categorical values, the trees seem to work better using numerical encoding. This can happen, one-hot encoding with many labels (37 in this case) leads to many more features which can complicate training the model.

## XGBoost

XGBoost is also based on decision trees. Instead of creating N trees that fit on subsets of data, the ``n_estimators`` trees are created sequentally. When a new tree is created, it tries to correct the errors of the previous trees by minimizing the error using gradient descent. The ``learning_rate`` parameter controls the influence of previous trees on the current tree. The ``subsample`` parameter controls show much data each tree uses for fitting. 

Again, we use cross-validation to find the best parameters. Given the compute limitation, we do a randomized search which tests only 50 parameter combinations. The xgboost library now handles categorical data (which applies one-hot encoding internally), so we don't need to do separate tests.

In [0]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
import mlflow
from sklearn.metrics import root_mean_squared_error, r2_score

# This version of XGBoost handles categorical data (with non-strings only), so we don't need one-hot encoding
target_col = "TOE_HAB"
feature_cols = ["country_encoded", "year"]

# This version of XGBoost does not like integers
for df_i in [df_train, df_test]:
    df_i["country_encoded"] = df_i["country_encoded"].astype("double")
    df_i["year"] = df_i["year"].astype("double")

X, y = df_train[feature_cols], df_train[target_col]

# Do a simple random seach to find the best parameters, instead of grid search because of limited resources
params_search = {
    'n_estimators': [50, 100, 300],
    'max_depth': [3, 6, 9], # smaller trees work better in XGBoost
    "learning_rate": [0.05, 0.1, 0.3],
    "subsample": [0.5, 0.75, 1]
}

# Use time series splits for temporally correlated data
tscv = TimeSeriesSplit(n_splits=5)

print()
param_search = RandomizedSearchCV(xgb.XGBRegressor(enable_categorical=True), 
                                  params_search, 
                                  n_iter=50, 
                                  cv=tscv, 
                                  scoring="neg_root_mean_squared_error", 
                                  verbose=True)
param_search.fit(X_train, y_train)
print("Best Parameters:", param_search.best_params_)
max_depth = param_search.best_params_["max_depth"]
n_estimators = param_search.best_params_["n_estimators"]
learning_rate = param_search.best_params_["learning_rate"]
subsample = param_search.best_params_["subsample"]

# Prepare train and test data
X_train, y_train = df_train[feature_cols], df_train[target_col]
X_test, y_test = df_test[feature_cols], df_test[target_col]
print(f"\nTraining data shapes -> X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")

# Train XGBoostRegressor
print(f"\nTraining XGBoost")
with mlflow.start_run(run_name="xgboost_regressor"):

    # Define model
    xgb_model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        enable_categorical=True,
        eval_metric="rmse",
        early_stopping_rounds=5, # Stop if the validation RMSE does not improve in 5 steps
        random_state=42
    )

    # Train model
    eval_set = [(X_train, y_train), (X_test,y_test)]
    xgb_model.fit(X_train, y_train, eval_set=eval_set)

    # Predict and measure performance
    y_pred = xgb_model.predict(X_test)
    results = xgb_model.evals_result()
    rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Log with MLflow
    for i, (train_rmse, val_rmse) in enumerate(zip(results["validation_0"]["rmse"], results["validation_1"]["rmse"])):
        mlflow.log_metric("train_rmse", train_rmse, step=i)
        mlflow.log_metric("val_rmse", val_rmse, step=i)
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("subsample", subsample)
    mlflow.log_param("random_state", 42)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)

    mlflow.xgboost.log_model(xgb_model, artifact_path="xgboost_regressor", input_example=X_train[0:1], model_format='ubj')

    print(f"RMSE: {rmse:.3f}, R2: {r2:.3f}")


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters: {'subsample': 1, 'n_estimators': 300, 'max_depth': 6, 'learning_rate': 0.3}

Training data shapes -> X_train: (681, 2), y_train: (681,), X_test: (132, 2), y_test: (132,)

Training XGBoost
[0]	validation_0-rmse:1.26746	validation_1-rmse:1.02916
[1]	validation_0-rmse:0.97125	validation_1-rmse:0.75436
[2]	validation_0-rmse:0.76919	validation_1-rmse:0.60292
[3]	validation_0-rmse:0.59383	validation_1-rmse:0.46718
[4]	validation_0-rmse:0.49999	validation_1-rmse:0.40849
[5]	validation_0-rmse:0.40367	validation_1-rmse:0.34629
[6]	validation_0-rmse:0.33433	validation_1-rmse:0.30144
[7]	validation_0-rmse:0.28770	validation_1-rmse:0.28991
[8]	validation_0-rmse:0.24619	validation_1-rmse:0.26525
[9]	validation_0-rmse:0.21208	validation_1-rmse:0.25950
[10]	validation_0-rmse:0.19201	validation_1-rmse:0.25288
[11]	validation_0-rmse:0.16423	validation_1-rmse:0.25013
[12]	validation_0-rmse:0.14712	validation_1-rmse:0.24242
[

## Neural networks

Neural networks are composed of interconnected artificial neurons. An activation function associated to a weight is usually used to represent the artificial neuron. The neural network is typically divided into layers, with each layer having a specific number of neurons (the size). The size of the input/output layer is the number of input/output features. The rest of the layers are called the hidden layers, which can have any number of neurons. The input value to the neural network is propagated through the neurons of the layers, leading to an output on the final layer. To train the neural network, the weights are adjusted based on the prediction error using algorithms like gradient descent.

In this case, we create our own custom PyTorch neural network with a custom number of layers and sizes. To deal with the country categorical input we can use embeddings, which maps the categorical value to a vector of numerical values. These output values are also learned while training the neural network. We can also introduce an optional dropout layer between the hidden layers, which will randomly deactivate neurons to prevent overfitting. We can tune the parameters implementing a custom Cross-validation method.

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib
import mlflow.pyfunc
import mlflow
import numpy as np
import random

# Seed the libraries for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Neural network class
class Net(nn.Module):
    def __init__(self, 
                 hidden_sizes=(32, 32), 
                 dropout=0.2,
                 country_idx=0,
                 year_idx=1,
                 n_countries=None, 
                 embed_dim=4):
        super(Net, self).__init__()

        # Columns where year and country are
        self.year_idx, self.country_idx = year_idx, country_idx

        # Embeddings for the countries
        self.country_embed = nn.Embedding(n_countries, embed_dim)

        # Layer setup
        hidden_sizes = [embed_dim+1] + list(hidden_sizes)
        layers = []
        for i in range(len(hidden_sizes)-1):
            layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
            layers.append(nn.ReLU())
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
        layers.append(nn.Linear(hidden_sizes[-1], 1))
        self.fc_layers = nn.Sequential(*layers)
    
    def forward(self, x):
        """
            Predicts the TOE_HAB/MTOE given the country and year.
            x: [country (encoded int), year (standarized float32)]
            output: predicted standarized TOE_HAB/MTOE
        """
        x = torch.cat([x[:,self.year_idx].reshape(-1,1), self.country_embed(x[:,self.country_idx].long())], dim=1)
        return self.fc_layers(x)


# Neural network model class
class NNModel(mlflow.pyfunc.PythonModel):
    def __init__(self, 
                 hidden_sizes=(32, 32),
                 lr=0.01,
                 dropout=0.2,
                 n_countries=37,
                 embed_dim=4):
        
        # Custom neural network
        self.net = Net(hidden_sizes=hidden_sizes,
                         dropout=dropout,
                         n_countries=n_countries,
                         embed_dim=embed_dim)
        
        # Scalers for standarizing the input and output values
        self.year_scaler, self.output_scaler = StandardScaler(), StandardScaler()
        self.country_idx, self.year_idx = 0, 1

        # Optimizer and loss function for training
        self.optimizer = optim.Adam(self.net.parameters(), lr=lr)
        self.mse = nn.MSELoss()

    def _prepare_training_data(self, X, y):
        """ Prepares the training data for the neural network.
            X: pd.DataFrame[country (int), year (int)].
            y: pd.DataFrame[TOE_HAB/MTOE (float)].
            output: np.array(X), np.array(y)
        """
        # Prepare data
        assert type(X) is pd.DataFrame and type(y) is pd.DataFrame, "Train data must be DataFrames"
        assert "country_encoded" in X.columns,  "X DataFrame must contain country_encoded column"
        assert "year" in X.columns, "X DataFrame must contain year column"
        self.year_scaler.fit(X["year"].values.reshape(-1,1))
        self.output_scaler.fit(y.values)
        X["year"] = self.year_scaler.transform(X["year"].values.reshape(-1,1))
        X = torch.tensor(X.values, dtype=torch.float32)
        y = torch.tensor(self.output_scaler.transform(y.values), dtype=torch.float32)
        return X, y

    def fit(self, X, y, epochs=100, test_frac=0, mlflow_run=False, verbose=False):
        """ 
            Trains the neural network.
            X: pd.DataFrame[country (int), year (int)].
            y: pd.DataFrame[TOE_HAB/MTOE (float)].
        """
        X, y = self._prepare_training_data(X.copy(), y.copy())
        
        if test_frac > 0:
            n_test = int(test_frac*len(X))
            X_train, X_test = X[:-n_test], X[-n_test:]
            y_train, y_test = y[:-n_test], y[-n_test:]
        else:
            X_train, y_train =  X, y

        # Train loop
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            y_hat = self.net(X_train)
            loss = self.mse(y_hat, y_train)
            loss.backward()
            self.optimizer.step()

            train_loss = loss.item()

            # If there is test data, compute test MSE
            if test_frac > 0:
                y_hat = self.net(X_test)
                val_loss = self.mse(y_hat, y_test).item()
                if mlflow_run:
                    mlflow.log_metrics({"train_mse": train_loss, "test_mse": val_loss}, step=epoch+1)
                if (epoch+1) % 10 == 0 and verbose:
                    print(f"Epoch {epoch+1}, Train loss: {train_loss:.4f}, Test loss: {val_loss:.4f}")
            else:
                if mlflow_run:
                    mlflow.log_metric("train_mse", train_loss, step=epoch+1)
                if (epoch+1) % 10 == 0 and verbose:
                    print(f"Epoch {epoch+1}, Train loss: {train_loss:.4f}")

        self.net.eval()

    def predict(self, model_input: pd.DataFrame) -> pd.Series:
        """
            Returns the predicted TOE_HAB/MTOE for the given input.
            model_input: pd.DataFrame[country (encoded int), year (int)].
            output: predicted pd.Series[TOE_HAB/MTOE (float)].
        """
        model_input = model_input.copy()
        model_input["year"] = self.year_scaler.transform(model_input["year"].values.reshape(-1,1))
        model_input = torch.tensor(model_input.values, dtype=torch.float32)
        with torch.no_grad():
            y_hat = self.net(model_input)
        y_hat = self.output_scaler.inverse_transform(y_hat)
        return pd.Series(y_hat.reshape(-1))

    def evaluate_rmse(self, X,  y):
        """
            Returns the RMSE of the model on the given data.
            X: [country (encoded int), year (int)]. 
            y: [TOE_HAB/MTOE].
            output: RMSE.
        """
        assert type(X) is pd.DataFrame and type(y) is pd.DataFrame, "Data must be DataFrames"
        assert "country_encoded" in X.columns,  "X DataFrame must contain country_encoded column"
        assert "year" in X.columns, "X DataFrame must contain year column"
        with torch.no_grad():
            y_pred = self.predict(X.copy())
        return np.sqrt(np.mean((y_pred.values - y.values.reshape(-1))**2))
        

# Neural networks also benefit from one-hot encoding, however we can use an embedding layer for memory efficiency
target_col = ["TOE_HAB"]
feature_cols = ["country_encoded", "year"]

# Prepare train and test data
X_train = df_train[feature_cols].astype("float32")
X_test = df_test[feature_cols].astype("float32")
y_train = df_train[target_col].astype("float32")
y_test = df_test[target_col].astype("float32")
n_countries = len(df["country"].unique().tolist())
print(f"Data shapes:\n X_train: {X_train.shape}\n y_train: {y_train.shape}\n X_test: {X_test.shape}\n y_test: {y_test.shape}")

# Tune parameters using cross-validation with a randomized search
print("\nTuning parameters")
from sklearn.model_selection import TimeSeriesSplit, ParameterGrid
param_grid = {
    "hidden_sizes": [(16,), (32,), (16, 16), (32, 32)],
    "lr": [0.005, 0.01, 0.05],
    "dropout": [0, 0.1, 0.2],
    "fit_epochs": [50, 100, 150, 200]}
param_grid = list(ParameterGrid(param_grid))
param_grid = np.random.choice(param_grid, 50, replace=False)
tscv = TimeSeriesSplit(n_splits=5) # Time splits

# Cross-validation
best_rmse = np.inf
best_idx = 0
for i, params in enumerate(param_grid):
    rmses = []
    for tr_index, val_index in tscv.split(X_train):
        X_tr, X_val = X_train.iloc[tr_index], X_train.iloc[val_index]
        y_tr, y_val = y_train.iloc[tr_index], y_train.iloc[val_index]
        model = NNModel(hidden_sizes=params["hidden_sizes"],
                        lr=params["lr"],
                        dropout=params["dropout"], 
                        n_countries=n_countries,
                        embed_dim=4)
        model.fit(X_tr, y_tr, epochs=params["fit_epochs"], test_frac=0, mlflow_run=False, verbose=False)
        rmses.append(model.evaluate_rmse(X_val, y_val))
    rmse = np.mean(rmses)
    if rmse < best_rmse:
        best_rmse = rmse
        best_idx = i
    if i%10 == 0:
        print(f"Configurations tried: {i}/{len(param_grid)} - Best RMSE: {best_rmse:.4f}")
    rmses = []
print(f"Best RMSE: {best_rmse:.4f}, at index {best_idx}")
params = param_grid[best_idx]
print("Best params:\n", params)

print("\nTraining neural network")
with mlflow.start_run(run_name="neural_network"):

    # Initialize neural network model with best parameters found
    model = NNModel(hidden_sizes=params["hidden_sizes"],
                lr=params["lr"],
                dropout=params["dropout"], 
                n_countries=n_countries,
                embed_dim=4)
    
    # Train model
    model.fit(X_train, y_train, epochs=params["fit_epochs"], test_frac=0.05, mlflow_run=True, verbose=True)

    # Evaluate model on test data
    rmse = model.evaluate_rmse(X_test, y_test)
    print("RMSE:", rmse)

    # Log with MLFlow
    mlflow.log_params(params)
    mlflow.log_metric("rmse", rmse)
    print("\nSaving neural network model")
    mlflow.pyfunc.log_model("neural_network", 
                            python_model=model, 
                            input_example=X_train.iloc[0:1],
                            )

Data shapes:
 X_train: (681, 2)
 y_train: (681, 1)
 X_test: (132, 2)
 y_test: (132, 1)

Tuning parameters
Configurations tried: 0/50 - Best RMSE: 0.4834
Configurations tried: 10/50 - Best RMSE: 0.3562
Configurations tried: 20/50 - Best RMSE: 0.3410
Configurations tried: 30/50 - Best RMSE: 0.3021
Configurations tried: 40/50 - Best RMSE: 0.3021
Best RMSE: 0.2942, at index 49
Best params:
 {'dropout': 0, 'fit_epochs': 100, 'hidden_sizes': (32, 32), 'lr': 0.05}

Training neural network
Epoch 10, Train loss: 0.5750, Test loss: 0.5398
Epoch 20, Train loss: 0.1453, Test loss: 0.1317
Epoch 30, Train loss: 0.0603, Test loss: 0.0422
Epoch 40, Train loss: 0.0289, Test loss: 0.0178
Epoch 50, Train loss: 0.0087, Test loss: 0.0252
Epoch 60, Train loss: 0.0059, Test loss: 0.0210
Epoch 70, Train loss: 0.0047, Test loss: 0.0173
Epoch 80, Train loss: 0.0043, Test loss: 0.0159
Epoch 90, Train loss: 0.0040, Test loss: 0.0171


2025/09/30 18:22:43 INFO mlflow.pyfunc: Inferring model signature from input example


Epoch 100, Train loss: 0.0038, Test loss: 0.0179
RMSE: 0.20443028264007856

Saving neural network model


# Predict with trained model

The two best models are the XGBoost and neural network ones, with a RMSE of ~0.24 and ~0.20 respectively. In this case, we will choose to continue with the neural network model, which achieved a slightly improved RMSE. The final model will be trained in the ``energy_nn_training`` notebook. The following cell shows the predicted TOE for Spain for the years 2000-2022 compared to the ground truth. Unlike with XGBoost, the predicted values of the neural network form a smoother trend curve, and the model does not overfit to the training data. Unfortunately, it cannot predict outliers like in 2020 due to covid.

In [0]:
# Prepare validation data, in this case data for Spain
df_val = df[["country", "country_encoded", "year", "TOE_HAB"]].copy()
df_val = df_val[df_val["country"] == "ES"]
X_val = df_val[["country_encoded", "year"]]

# Predict with XGBoost and neural network
y_pred_xgb = xgb_model.predict(X_val)
with torch.no_grad():
    y_pred_nn = model.predict(X_val)

# Visualize predictions
df_val["xgb_predicted_TOE_HAB"] = y_pred_xgb.reshape(-1,1)
df_val["nn_predicted_TOE_HAB"] = y_pred_nn.values
display(df_val)

country,country_encoded,year,TOE_HAB,xgb_predicted_TOE_HAB,nn_predicted_TOE_HAB
ES,11,2000,1.97,2.0543656,2.0321082651449087
ES,11,2001,2.06,2.0543656,2.053221288144897
ES,11,2002,2.06,2.0543656,2.0737591265160837
ES,11,2003,2.15,2.1226888,2.092308136187962
ES,11,2004,2.21,2.1226888,2.100087834319733
ES,11,2005,2.25,2.1226888,2.1059940710855933
ES,11,2006,2.16,2.1226888,2.0982163253625887
ES,11,2007,2.18,2.1226888,2.074491830482935
ES,11,2008,2.07,2.1226888,2.041703352997227
ES,11,2009,1.9,1.9708691,2.001392870306051


Databricks visualization. Run in Databricks to view.